<a href="https://colab.research.google.com/github/ryanstwrt/osu-transport/blob/gh-pages/osu-transport/users/stewryan/Blackboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Blackboard System
Below is a basic blackboard system which involved three agents; a controller and two knowledge agents.
The blackboard is used to create a HDF5 file, which the agents will write too, and keep track of the progress of the agents.
The controller agent initiates the problem and tells agents A and B to add their portions of the problem to the blackboard.
The simple example has agents A and B writing the square of values 0 to 9 to the database.
Neither agent wants to copy what the other is dointn so they perform a simple check to ensure they are producing unique data. If so, they write the square of the given value to the database.
Agent A is working in forward from 0 to 9, while Agent B is working in reverse. At the end, the database should have 10 entries ranging from 0 to 9.

In [27]:
pip install osbrain

In [0]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import Agent
import time
import tables as tb
import os
import numpy as np

In [36]:
class Blackboard(object):
  """This class will be the blackboard from which knowledge agents write to and read from.
  The state variable keeps track of who is writting to the blackboard, and how many problems have been solved."""

  def __init__(self):
    self.agents = []
    self.state = {
        'problems': 0,
        'contributions': [],
    }
    #Set up the HDF5 databsase
    db = tb.open_file("blackboard_db", "w")
    db.close()

  def add_agent(self, agent):
    self.agents.append(agent)

class Controller(object):
  """Controls the flow of work distribution in the problem"""

  def __init__(self, blackboard):
    self.blackboard = blackboard

  def solve_problem(self):
    i = 0
    while self.blackboard.state['problems'] < 20:
      for agent in self.blackboard.agents:
        agent.contribute(i)
      i+=1        
    return self.blackboard.state['contributions']
  

class baseAgent(object):
  """Base agent to define __init__ and basic functions"""

  def __init__(self, blackboard, name):
    self.blackboard = blackboard
    self.name = name
    self.root = '/' + self.name
    db = tb.open_file("blackboard_db", "a")
    db.create_group(db.root, self.name)    
    db.close()

  def contribute(self, i):
    pass

  def squared(self, x):
    return x * x

class AgentA(baseAgent):
  """Agent A will count from 0 to 10"""

  def contribute(self, i):
    doubled = self.squared(i)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_{}'.format(str(i)) in node:
          self.blackboard.state['problems'] = 20
      else:
        db.create_array(self.root, 'Val_{}'.format(str(i)) , [doubled])
    self.blackboard.state['problems'] += 1
    self.blackboard.state['contributions'] += [self.__class__.__name__]

class AgentB(baseAgent):
  """Agent B will count in reverse from 10 to 0"""

  def contribute(self, i):
    i = 9 - i
    doubled = self.squared(i)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_{}'.format(str(i)) in node:
          self.blackboard.state['problems'] = 20
      else:
        db.create_array(self.root, 'Val_{}'.format(str(i)) , [doubled])
    self.blackboard.state['problems'] += 1
    self.blackboard.state['contributions'] += [self.__class__.__name__]

if __name__ == '__main__':
  os.system('rm blackboard_db')
  blackboard = Blackboard()
  blackboard.add_agent(AgentA(blackboard, 'AgentA'))
  blackboard.add_agent(AgentB(blackboard, 'AgentB'))
  c = Controller(blackboard)
  contributions = c.solve_problem()

  h5db = tb.open_file('blackboard_db', 'r+')
  print(h5db)
  h5db.close()

blackboard_db (File) ''
Last modif.: 'Thu Oct 31 13:39:57 2019'
Object Tree: 
/ (RootGroup) ''
/AgentA (Group) ''
/AgentA/Val_0 (Array(1,)) ''
/AgentA/Val_1 (Array(1,)) ''
/AgentA/Val_2 (Array(1,)) ''
/AgentA/Val_3 (Array(1,)) ''
/AgentA/Val_4 (Array(1,)) ''
/AgentA/Val_5 (Array(1,)) ''
/AgentB (Group) ''
/AgentB/Val_4 (Array(1,)) ''
/AgentB/Val_5 (Array(1,)) ''
/AgentB/Val_6 (Array(1,)) ''
/AgentB/Val_7 (Array(1,)) ''
/AgentB/Val_8 (Array(1,)) ''
/AgentB/Val_9 (Array(1,)) ''



In [0]:
import tables as tb
import os

class Blackboard(object):
  """This class will be the blackboard from which knowledge agents write to and read from."""

  def __init__(self):
    self.agents = []
    self.state = {
        'problems': 0,
        'contributions': [],
    }
    #Set up the HDF5 databsase
    db = tb.open_file("blackboard_db", "w")
    db.close()

  def add_agent(self, agent):
    self.agents.append(agent)


class CA(Agent):
  def on_init(self):
    self.bind('PUSH', alias='main')
  
  def solve_problem(self):
    print(self.blackboard)
    """Controls the flow of work distribution in the problem  
    Agent should meet in the middle and stop running"""

    i = 0
    while i < 5:
      for agent in self.blackboard.agents:
        agent.contribute(i)
      i+=1        
    return self.blackboard.state['contributions']
    
class baseKA(Agent):
  """Base agent to define __init__ and basic functions"""
  
  def on_init(self):
    self.set_attr(root = '/{}'.format(self.name))
    db = tb.open_file("blackboard_db", "a")
    db.create_group(db.root, self.name)    
    db.close()

  def contribute(self, i):
    pass

  def squared(self, x):
    return x * x

class KA_A(baseKA):
  """Agent A will count from 0 to 10"""

  def contribute(self, i):
    doubled = self.squared(i)
    print(self.blackboard)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_' + str(i) in node:
          self.blackboard.state['problems'] = 20
          break
      else:
        db.create_array(self.root, 'Val_' + str(i), [doubled])
    print('test')
    self.blackboard.state['problems'] += 1
    self.blackboard.state['contributions'] += [self.__class__.__name__]

class KA_B(baseKA):
  """Agent B will count in reverse from 10 to 0"""

  def contribute(self, i):
    print(self.blackboard)
    i = 10 - i
    doubled = self.squared(i)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_' + str(i) in node:
          self.blackboard.state['problems'] = 20
          break
      else:
        db.create_array(self.root, 'Val_' + str(i), [doubled])
    self.blackboard.state['problems'] += 1
    self.blackboard.state['contributions'] += [self.__class__.__name__]

if __name__ == '__main__':
  os.system('rm blackboard_db')
  #Initialize the MAS and the blackboard
  ns = run_nameserver()
  blackboard = Blackboard()
 
  #Initialize the knowledge agents
  KA_A = run_agent(name='KA_A', base=KA_A, attributes={'blackboard': blackboard})
  KA_B = run_agent(name='KA_B', base=KA_B, attributes={'blackboard': blackboard})

  #Add the knowledge agents to the blackboard
  blackboard.add_agent(KA_A)
  blackboard.add_agent(KA_B)

  #Initialize the controller agent and run the problem
  CA = run_agent(name='CA', base=CA, attributes={'blackboard': blackboard})
  contributions = CA.solve_problem()
  print(blackboard.state)
  print(contributions)
  ns.shutdown()


  #h5db = tb.open_file('blackboard_db', 'r+')
  #print(h5db)
  #h5db.close()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:13960 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:13960
test
test
test
test
test
{'problems': 0, 'contributions': []}
[]
NS shut down.


# Multi-Agent Blackboard System (MABS)
## Base Classes
The following section defines our base classes, which won't change much.
The blackboard class will act as both the repository for information obtained by the agents, and as the controller for the flow of agent work.
Combining the controller agent and the blackboard allows for fewer calls to be made between different agents.

In [0]:
def log_message(self, message):
  self.log_info('Recieved: {}'.format(message))

def permission_granted(self, message):
  return message

class Blackboard(Agent):
  def on_init(self):
    self.agents = []
    self.state = {
        'problems': 0,
        'contributions': [],
    }
    db = tb.open_file("blackboard_db", "w")
    db.close()

  def solve_problem(self):
    i = 0
    while i < 2:
      for agent in self.agents:
        agent.simulate(i)
      i+=1        
    return self.state['contributions']

  def add_agent(self, agent):
    self.agents.append(agent)

class baseKA(Agent):
  """Base agent to define __init__ and basic functions"""
  
  def on_init(self):
    self.bind('REP', alias='ask_permission', handler='ask_permission')
    self.root = '/{}'.format(self.name)
    self.db_entry = []
    db = tb.open_file("blackboard_db", "a")
    db.create_group(db.root, self.name)    
    db.close()

  def simulate(self, i):
    pass

  def write_to_db(self):
    pass

  def ask_permission(self, message):
    print(message)
    self.write_to_db()

  def squared(self, x):
    return x * x

In [57]:
class KA_A(baseKA):

  def simulate(self, i):
    doubled = self.squared(i)

  def write_to_db(self):
    print('at writing step')
    time.sleep(2)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_' + str(i) in node:
          break
      else:
        db.create_array(self.root, 'Val_' + str(i), [doubled])

if __name__ == '__main__':
  os.system('rm blackboard_db')
  ns = run_nameserver()
  bb = run_agent(name='blackboard', base=Blackboard)
  ka_a = run_agent(name='ka_a', base=KA_A)
  bb.add_agent(ka_a)
  #ka_a.connect(bb.addr('permission'), handler=log_message)
  bb.connect(ka_a.addr('ask_permission'), alias='ask_permission')

  a = bb.send('ask_permission', True)
  time.sleep(2)
  bb.recv('ask_permission')

  contributions = bb.solve_problem()
  h5db = tb.open_file('blackboard_db', 'r+')
  print(h5db)
  h5db.close()
  ns.shutdown()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:10920 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:10920
True
at writing step


ERROR [2019-10-31 14:01:02.969192] (ka_a): An exception occurred while running! (name 'i' is not defined)

|>>>>>>>>Traceback (most recent call last):
|   File "/usr/local/lib/python3.6/dist-packages/osbrain/agent.py", line 1680, in run
|     self._loop()
|   File "/usr/local/lib/python3.6/dist-packages/osbrain/agent.py", line 1208, in _loop
|     if self._iterate():
|   File "/usr/local/lib/python3.6/dist-packages/osbrain/agent.py", line 1245, in _iterate
|     self._process_events(events)
|   File "/usr/local/lib/python3.6/dist-packages/osbrain/agent.py", line 1282, in _process_events
|     self._process_single_event(socket)
|   File "/usr/local/lib/python3.6/dist-packages/osbrain/agent.py", line 1300, in _process_single_event
|     self._process_rep_event(socket, address, data)
|   File "/usr/local/lib/python3.6/dist-packages/osbrain/agent.py", line 1344, in _process_rep_event
|     reply = handler(self, message)
|   File "<ipython-input-56-ad049ede2b8a>", line 47, in ask_permission

KeyboardInterrupt: ignored

NS shut down.
